Accuracy 🎯 vs Performance 🏃 
---
https://github.com/jeffmur/fhe-video-similarity/wiki/experiment

This notebook serves to analyze, compare, and visulize the trade-off between accuracy and performance when using Fully Homomorphic Encryption (FHE) to compute video similarity scores.

Note: Every experiment uses the same encryption scheme & parameters:

* Cryptosystem: CKKS
* Polynomial Degree: 4096
* Encode Scalar: 2^40
* qSizes: [60, 40, 40, 60]

There are three metrics being gather within the application:

⚙️ **Pre-processing Time**: The time it takes to convert the video into a format that can be used for comparison.

📊 **Similarity Scores**: The time it takes to encrypt & compute a similarity score.

In [1]:
from utils.tables import verbose_md_table, mean_md_table

# Experiment 1: Static Color

Using ffmpeg to generate static colored videos with no audio.

## Scenario 1: Black & White

Compare absolute black and white videos to determine the similarity score between the two.

### Test 1: 10 second video

```bash
ffmpeg -f lavfi -i color=c=black:s=1280x720 -c:v libx264 -t 10 -pix_fmt yuv420p -an Black_720p.mp4
ffmpeg -f lavfi -i color=c=white:s=1280x720 -c:v libx264 -t 10 -pix_fmt yuv420p -an White_720p.mp4
```

#### Assertion 1: 100% Similarity

> Hypothesis: The score should indicate high similarity between the two videos, as the segment distribution will not change.

In [2]:
black_white_10s = "1_Static_Color/1_Black_White/1_10s/1_100_percent"
verbose_md_table(black_white_10s)

Similarity (frameCount) | Score [%] | Mean FHE Absolute Error | Pre-processing (s) | Encryption (ms) | FHE Compute (ms) | Plaintext Compute (ms) | FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---|---|---
KLD (all) | 4.04e-11 [100.00%] | 8.08e-11 | 2.00 | 25.00 | 9.00 | 0.33 | 8.67 [2602.70%]
Cramer (all) | 4.96e-10 [100.00%] | 9.92e-10 | 2.00 | 11.00 | 19.00 | 0.19 | 18.81 [9744.56%]
BC (all) | 1.00e+00 [100.00%] | 6.88e-10 | 2.00 | 24.00 | 5.00 | 0.09 | 4.91 [5713.95%]
KLD (firstLast) | 2.07e-10 [100.00%] | 4.14e-10 | 1.29 | 25.00 | 9.00 | 0.36 | 8.64 [2393.07%]
Cramer (firstLast) | 9.33e-10 [100.00%] | 1.87e-09 | 1.29 | 12.00 | 19.00 | 0.15 | 18.84 [12158.06%]
BC (firstLast) | 1.00e+00 [100.00%] | 2.68e-10 | 1.29 | 24.00 | 5.00 | 0.08 | 4.92 [6229.11%]
KLD (randomHalf) | 3.35e-01 [74.91%] | 5.36e-10 | 1.62 | 23.00 | 7.00 | 0.03 | 6.97 [26823.08%]
Cramer (randomHalf) | 2.31e-01 [91.73%] | 3.15e-10 | 1.62 | 12.00 | 19.00 | 0.02 | 18.98 [90376.19%]
BC (randomHalf) | 9.17e-01 [76.95%] | 1.85e-10 | 1.62 | 23.00 | 4.00 | 0.02 | 3.98 [26566.67%]

In [3]:
mean_md_table(black_white_10s)


Similarity | Mean Score [%] | Mean FHE Mean Absolute Error | Mean Pre-processing (s) | Mean Encryption (ms) | Mean FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---
KLD | 1.12e-01 [89.96%] | 3.44e-10 | 1.64 | 24.33 | 8.09 [3372.22%]
Cramer | 7.68e-02 [97.24%] | 1.06e-09 | 1.64 | 11.67 | 18.88 [15347.15%]
BC | 9.72e-01 [92.32%] | 3.81e-10 | 1.64 | 23.67 | 4.61 [7677.78%]


# Experiment 2: RGB Color from YouTube

## Scenario 1: Fade Single Color

### Test 1: Alignment

#### Assertion 1: First 14s vs. Next 14s

In [4]:
rgb_fade_single_14s = "2_YT_RGB/1_Single_Fade/1_Alignment/1_First_14s_Next_14s"
verbose_md_table(rgb_fade_single_14s)

Similarity (frameCount) | Score [%] | Mean FHE Absolute Error | Pre-processing (s) | Encryption (ms) | FHE Compute (ms) | Plaintext Compute (ms) | FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---|---|---
KLD (all) | 5.69e-03 [99.43%] | 5.65e-10 | 6.00 | 33.00 | 10.00 | 0.05 | 9.95 [20733.33%]
Cramer (all) | 4.43e-02 [99.86%] | 8.03e-10 | 6.00 | 16.00 | 25.00 | 0.02 | 24.98 [118947.62%]
BC (all) | 9.99e-01 [95.57%] | 1.50e-09 | 6.00 | 33.00 | 6.00 | 0.01 | 5.99 [74900.00%]
KLD (firstLast) | 2.09e-02 [97.96%] | 2.86e-10 | 1.94 | 33.00 | 11.00 | 0.35 | 10.65 [3033.90%]
Cramer (firstLast) | 9.52e-02 [99.47%] | 3.31e-10 | 1.94 | 16.00 | 26.00 | 0.16 | 25.84 [15949.38%]
BC (firstLast) | 9.95e-01 [90.48%] | 1.53e-10 | 1.94 | 33.00 | 6.00 | 0.08 | 5.92 [7042.86%]
KLD (randomHalf) | 5.81e-01 [63.26%] | 3.08e-09 | 2.00 | 38.00 | 9.00 | 0.02 | 8.98 [39030.43%]
Cramer (randomHalf) | 4.37e-01 [88.16%] | 4.74e-10 | 2.00 | 16.00 | 25.00 | 0.02 | 24.98 [124900.00%]
BC (randomHalf) | 8.82e-01 [56.35%] | 4.57e-10 | 2.00 | 32.00 | 6.00 | 0.01 | 5.99 [59900.00%]

In [5]:
mean_md_table(rgb_fade_single_14s)


Similarity | Mean Score [%] | Mean FHE Mean Absolute Error | Mean Pre-processing (s) | Mean Encryption (ms) | Mean FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---
KLD | 2.02e-01 [83.16%] | 1.31e-09 | 3.31 | 34.67 | 9.86 [7009.00%]
Cramer | 1.92e-01 [95.83%] | 5.36e-10 | 3.31 | 16.00 | 25.27 [37338.42%]
BC | 9.58e-01 [80.80%] | 7.02e-10 | 3.31 | 32.67 | 5.97 [17547.06%]


#### Assertion 2: Offset by 2s of 14s segments

In [6]:
rgb_fade_single_offset_2s = "2_YT_RGB/1_Single_Fade/1_Alignment/2_Offset_2s_of_14s"
verbose_md_table(rgb_fade_single_offset_2s)

Similarity (frameCount) | Score [%] | Mean FHE Absolute Error | Pre-processing (s) | Encryption (ms) | FHE Compute (ms) | Plaintext Compute (ms) | FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---|---|---
KLD (all) | 3.32e-03 [99.67%] | 2.00e-10 | 6.00 | 32.00 | 9.00 | 0.02 | 8.98 [42757.14%]
Cramer (all) | 5.07e-02 [99.92%] | 7.98e-10 | 6.00 | 16.00 | 26.00 | 0.02 | 25.98 [129900.00%]
BC (all) | 9.99e-01 [94.93%] | 1.69e-10 | 6.00 | 32.00 | 6.00 | 0.01 | 5.99 [59900.00%]
KLD (firstLast) | 3.06e-02 [97.03%] | 4.46e-10 | 1.86 | 32.00 | 10.00 | 0.03 | 9.97 [38361.54%]
Cramer (firstLast) | 9.52e-02 [99.25%] | 4.15e-10 | 1.86 | 16.00 | 25.00 | 0.03 | 24.97 [83233.33%]
BC (firstLast) | 9.93e-01 [90.48%] | 1.28e-10 | 1.86 | 32.00 | 6.00 | 0.01 | 5.99 [49900.00%]
KLD (randomHalf) | 4.06e-01 [71.13%] | 2.24e-09 | 2.00 | 33.00 | 9.00 | 0.04 | 8.96 [21328.57%]
Cramer (randomHalf) | 4.06e-01 [91.45%] | 3.84e-10 | 2.00 | 18.00 | 27.00 | 0.02 | 26.98 [134900.00%]
BC (randomHalf) | 9.15e-01 [59.39%] | 1.47e-09 | 2.00 | 33.00 | 6.00 | 0.01 | 5.99 [49900.00%]

In [7]:
mean_md_table(rgb_fade_single_offset_2s)


Similarity | Mean Score [%] | Mean FHE Mean Absolute Error | Mean Pre-processing (s) | Mean Encryption (ms) | Mean FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---
KLD | 1.47e-01 [87.21%] | 9.63e-10 | 3.29 | 32.33 | 9.30 [31360.67%]
Cramer | 1.84e-01 [96.87%] | 5.32e-10 | 3.29 | 16.67 | 25.98 [111328.57%]
BC | 9.69e-01 [81.60%] | 5.90e-10 | 3.29 | 32.33 | 5.99 [52841.18%]


##### Assertion 3: First 7s vs. Next 7s

In [8]:
rgb_fade_single_7s = "2_YT_RGB/1_Single_Fade/1_Alignment/3_First_7s_Next_7s"
verbose_md_table(rgb_fade_single_7s)

Similarity (frameCount) | Score [%] | Mean FHE Absolute Error | Pre-processing (s) | Encryption (ms) | FHE Compute (ms) | Plaintext Compute (ms) | FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---|---|---
KLD (all) | 6.09e-03 [99.40%] | 1.77e-10 | 2.00 | 24.00 | 5.00 | 0.02 | 4.98 [27677.78%]
Cramer (all) | 9.18e-02 [99.85%] | 1.64e-10 | 2.00 | 8.00 | 12.00 | 0.05 | 11.95 [23900.00%]
BC (all) | 9.98e-01 [90.82%] | 3.56e-10 | 2.00 | 16.00 | 3.00 | 0.01 | 2.99 [33233.33%]
KLD (firstLast) | 2.31e-02 [97.74%] | 2.16e-10 | 1.14 | 16.00 | 5.00 | 0.02 | 4.98 [26215.79%]
Cramer (firstLast) | 1.38e-01 [99.43%] | 4.68e-10 | 1.14 | 8.00 | 13.00 | 0.02 | 12.98 [72122.22%]
BC (firstLast) | 9.94e-01 [86.23%] | 4.64e-10 | 1.14 | 16.00 | 3.00 | 0.01 | 2.99 [29900.00%]
KLD (randomHalf) | 3.16e-01 [75.96%] | 4.11e-09 | 1.61 | 16.00 | 4.00 | 0.03 | 3.97 [15284.62%]
Cramer (randomHalf) | 3.01e-01 [92.51%] | 3.89e-10 | 1.61 | 8.00 | 13.00 | 0.03 | 12.97 [48048.15%]
BC (randomHalf) | 9.25e-01 [69.93%] | 9.01e-12 | 1.61 | 16.00 | 3.00 | 0.01 | 2.99 [21328.57%]

In [9]:
mean_md_table(rgb_fade_single_7s)


Similarity | Mean Score [%] | Mean FHE Mean Absolute Error | Mean Pre-processing (s) | Mean Encryption (ms) | Mean FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---
KLD | 1.15e-01 [89.67%] | 1.50e-09 | 1.58 | 18.67 | 4.65 [22122.22%]
Cramer | 1.77e-01 [97.26%] | 3.40e-10 | 1.58 | 8.00 | 12.63 [39900.00%]
BC | 9.73e-01 [82.33%] | 2.76e-10 | 1.58 | 16.00 | 2.99 [27172.73%]


### Test 2: Compare to Static Color

##### Assertion 1: Black 10s

In [10]:
rgb_fade_single_10s_black = "2_YT_RGB/1_Single_Fade/2_Compare_Static/1_Black_10s"
verbose_md_table(rgb_fade_single_10s_black)

Similarity (frameCount) | Score [%] | Mean FHE Absolute Error | Pre-processing (s) | Encryption (ms) | FHE Compute (ms) | Plaintext Compute (ms) | FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---|---|---
KLD (all) | 2.65e-02 [97.42%] | 1.06e-09 | 3.00 | 28.00 | 8.00 | 0.02 | 7.98 [34682.61%]
Cramer (all) | 1.73e-01 [99.27%] | 1.01e-09 | 3.00 | 11.00 | 20.00 | 0.02 | 19.98 [99900.00%]
BC (all) | 9.93e-01 [82.73%] | 1.89e-10 | 3.00 | 24.00 | 5.00 | 0.01 | 4.99 [55455.56%]
KLD (firstLast) | 4.88e-02 [95.34%] | 3.76e-09 | 1.08 | 30.00 | 10.00 | 0.81 | 9.19 [1134.57%]
Cramer (firstLast) | 2.49e-01 [98.66%] | 1.07e-11 | 1.08 | 12.00 | 20.00 | 0.25 | 19.75 [7900.00%]
BC (firstLast) | 9.87e-01 [75.13%] | 5.36e-10 | 1.08 | 24.00 | 6.00 | 0.13 | 5.87 [4445.45%]
KLD (randomHalf) | 1.17e-01 [89.49%] | 2.79e-09 | 1.55 | 24.00 | 8.00 | 0.02 | 7.98 [33233.33%]
Cramer (randomHalf) | 3.64e-01 [97.03%] | 7.65e-10 | 1.55 | 12.00 | 19.00 | 0.03 | 18.97 [67757.14%]
BC (randomHalf) | 9.70e-01 [63.57%] | 6.42e-11 | 1.55 | 24.00 | 5.00 | 0.01 | 4.99 [55455.56%]

In [11]:
mean_md_table(rgb_fade_single_10s_black)


Similarity | Mean Score [%] | Mean FHE Mean Absolute Error | Mean Pre-processing (s) | Mean Encryption (ms) | Mean FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---
KLD | 6.43e-02 [93.96%] | 2.53e-09 | 1.87 | 27.33 | 8.38 [2933.84%]
Cramer | 2.62e-01 [98.32%] | 5.95e-10 | 1.87 | 11.67 | 19.57 [19698.66%]
BC | 9.83e-01 [73.81%] | 2.63e-10 | 1.87 | 24.00 | 5.28 [10566.67%]


##### Assertion 2: White 10s

In [12]:
rgb_fade_single_10s_white = "2_YT_RGB/1_Single_Fade/2_Compare_Static/2_White_10s"
verbose_md_table(rgb_fade_single_10s_white)

Similarity (frameCount) | Score [%] | Mean FHE Absolute Error | Pre-processing (s) | Encryption (ms) | FHE Compute (ms) | Plaintext Compute (ms) | FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---|---|---
KLD (all) | 2.06e-02 [97.98%] | 8.01e-10 | 4.00 | 64.00 | 19.00 | 0.06 | 18.94 [30545.16%]
Cramer (all) | 1.61e-01 [99.46%] | 1.63e-09 | 4.00 | 29.00 | 47.00 | 0.06 | 46.94 [83828.57%]
BC (all) | 9.95e-01 [83.90%] | 2.12e-09 | 4.00 | 56.00 | 12.00 | 0.03 | 11.97 [44344.44%]
KLD (firstLast) | 2.53e-02 [97.53%] | 5.48e-10 | 1.33 | 29.00 | 10.00 | 0.03 | 9.97 [32158.06%]
Cramer (firstLast) | 1.79e-01 [99.34%] | 7.06e-10 | 1.33 | 15.00 | 24.00 | 0.03 | 23.98 [95900.00%]
BC (firstLast) | 9.93e-01 [82.09%] | 1.32e-09 | 1.33 | 28.00 | 6.00 | 0.01 | 5.99 [49900.00%]
KLD (randomHalf) | 3.20e-02 [96.90%] | 8.21e-10 | 1.88 | 100.00 | 28.00 | 0.10 | 27.90 [28765.98%]
Cramer (randomHalf) | 1.46e-01 [99.17%] | 2.26e-09 | 1.88 | 43.00 | 69.00 | 0.08 | 68.92 [86150.00%]
BC (randomHalf) | 9.92e-01 [85.42%] | 5.54e-09 | 1.88 | 84.00 | 18.00 | 0.05 | 17.95 [35900.00%]

In [13]:
mean_md_table(rgb_fade_single_10s_white)


Similarity | Mean Score [%] | Mean FHE Mean Absolute Error | Mean Pre-processing (s) | Mean Encryption (ms) | Mean FHE/Plain Compute Growth (ms) [%]
---|---|---|---|---|---
KLD | 2.60e-02 [97.47%] | 7.23e-10 | 2.40 | 64.33 | 18.94 [29900.00%]
Cramer | 1.62e-01 [99.32%] | 1.53e-09 | 2.40 | 29.00 | 46.61 [86856.52%]
BC | 9.93e-01 [83.80%] | 2.99e-09 | 2.40 | 56.00 | 11.97 [40349.44%]
